In [1]:
import pandas as pd
import requests
import os
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the Excel file
df = pd.read_excel(r"C:\Users\amith\Kenpath\zavmo-api\docs\nos-ofqual\latest_nos_data.xlsx")

# Function to clean file names
def clean_filename(filename):
    """Clean a filename to remove invalid characters."""
    return re.sub(r'[\\/*?:"<>|]', "", filename).replace(" ", "_")

# Function to download a file
def download_file(row):
    nos_title = row['nos_title']
    pdf_link = row['pdf_link']

    if pd.notna(pdf_link):
        cleaned_title = clean_filename(nos_title)
        filename = os.path.join(download_directory, f"{cleaned_title}.pdf")

        try:
            response = requests.get(pdf_link, stream=True, timeout=15)
            response.raise_for_status()

            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            return (pdf_link, "Success")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading {pdf_link}: {e}")
            return (pdf_link, "Failed")

    return (None, "Skipped")

# Specify the download directory
download_directory = r"C:\Users\amith\Kenpath\NOS"
os.makedirs(download_directory, exist_ok=True)

# Set the number of threads (adjust based on system/network)
num_threads = 50  # You can increase this if your internet and system can handle it

# Use ThreadPoolExecutor for parallel downloads
results = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    future_to_url = {executor.submit(download_file, row): row for _, row in df.iterrows()}
    
    for future in as_completed(future_to_url):
        results.append(future.result())

# Save download status to an Excel file
result_df = pd.DataFrame(results, columns=["PDF_Link", "Status"])
result_df.to_excel(r"C:\Users\amith\Kenpath\NOS\download_status.xlsx", index=False)

print("All downloads completed. Status saved in 'download_status.xlsx'.")


Error downloading https://files.ukstandards.org.uk/pdfs/IMIARBG11.pdf: 404 Client Error: The specified blob does not exist. for url: https://files.ukstandards.org.uk/pdfs/IMIARBG11.pdf
Error downloading https://files.ukstandards.org.uk/pdfs/IMIARC07.pdf: 404 Client Error: The specified blob does not exist. for url: https://files.ukstandards.org.uk/pdfs/IMIARC07.pdf
Error downloading https://files.ukstandards.org.uk/pdfs/SEMAE3186L.pdf: 404 Client Error: The specified blob does not exist. for url: https://files.ukstandards.org.uk/pdfs/SEMAE3186L.pdf
Error downloading https://files.ukstandards.org.uk/pdfs/SKAG3_(3).pdf: 404 Client Error: The specified blob does not exist. for url: https://files.ukstandards.org.uk/pdfs/SKAG3_(3).pdf
Error downloading https://files.ukstandards.org.uk/pdfs/LANCS59.pdf: 404 Client Error: The specified blob does not exist. for url: https://files.ukstandards.org.uk/pdfs/LANCS59.pdf
Error downloading https://files.ukstandards.org.uk/pdfs/LANCS4L.pdf: 404 Client